In [1]:
import requests
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup

# Input Parameters

In [29]:
range_low = '35078000'
range_high = '35080000'
start_date = '1/08/2020'
end_date = '14/08/2020'

# Requesting Main Site

In [2]:
session = AsyncHTMLSession()

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
}

In [4]:
url = 'https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx'


In [5]:
response = await session.get(url, headers=headers)
response

<Response [200]>

# Storing HTML Response in file and Creating Soup Object

In [6]:
# Rendering JavScript

await response.html.arender()

In [7]:
html = response.html.html

In [8]:
soup = BeautifulSoup(html, 'lxml')

In [9]:
with open('Hacienda_DUAS_JS_Rendered.html', 'wb') as f:
    f.write(soup.prettify('utf-8'))

# Dealing with Captcha



## Obtaining Captcha Img Source

In [10]:
img_src = response.html.xpath('//div[@id="captchaImage"]/img')[0].attrs['src']

In [11]:
captcha_number = int(img_src.split('/')[-1].split('.')[0])

## Beating Captcha

### Pulling Captcha Answer from Database

In [12]:
import pandas as pd 

In [13]:
captchas = pd.read_csv('../Captchas/captchas.csv', index_col=0)

In [14]:
captcha_value = captchas.iloc[captcha_number - 1][0]

### Getting Ajax Token

In [15]:
token = soup.find_all('input')[-1].attrs['value'].split(',')[-10].split('"')[-2]

In [16]:
token 

'2517382EB5179B1D57DDA85B1A1E8EF8'

### POST Request with Captcha

In [17]:
form_data = {
    'BTNMONITOR_MPAGE': 'Monitoreo Sistema Tic@',
    'vVADUANA': 'X',
    'vVFCHIN': '1/08/2020',
    'vVFCHFN': '14/08/2020',
    'vVNCMINI': '35078000',
    'vVNCMFIN': '35080000',
    'vTIPO_REGI': 'I',
    'cfield': captcha_value,
    'MPW0015LOGIN': 'Login',
    'MPW0015BUTTON3': 'Desconectarse',
    'MPW0015BUTTON2': 'Nuevo Usuario'
}


post_url = 'https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx'

In [18]:
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'es,en-US;q=0.9,en;q=0.8',
    'AJAX_SECURITY_TOKEN': token,
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
}


In [19]:
response = await session.post(post_url, data=form_data, headers=headers)


In [20]:
#Rendering JS

await response.html.arender()

In [21]:
html = response.html.html

In [22]:
soup = BeautifulSoup(html, 'lxml')

In [23]:
with open('After_POST_Request.html', 'wb') as f:
    f.write(soup.prettify('utf-8'))

In [24]:
response

<Response [200]>

In [25]:
response.html.xpath('//table[@id="SubfileContainerTbl"]/tbody')

[<Element 'tbody' >]

In [26]:
response.url

'https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx'

In [27]:
session.close()

<coroutine object AsyncHTMLSession.close at 0x7f45e9339c20>

In [28]:
import dryscrape